# Task 3

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tabulate import tabulate

csv_files = {
    'Normal': r'C:\Users\prags\Desktop\hackathon\Normal_statistics.csv', 'Complex_Partial_Seizures':r'C:\Users\prags\Desktop\hackathon\Complex_Partial_Seizures_statistics.csv', 'Electrographic_Seizures': r'C:\Users\prags\Desktop\hackathon\Electrographic_Seizures_statistics.csv', 'Video_detected_Seizures_with_no_visual_change_over_EEG': r'C:\Users\prags\Desktop\hackathon\Video_detected_Seizures_with_no_visual_change_over_EEG_statistics.csv', }

all_features = []
all_labels = []

for label, file_path in csv_files.items():
    data = pd.read_csv(file_path)
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)
    data_imputed_df = pd.DataFrame(data_imputed, columns=data.columns)
    all_features.append(data_imputed_df)
    labels = [label] * len(data_imputed_df)
    all_labels.extend(labels)

X = pd.concat(all_features, ignore_index=True)
y = pd.Series(all_labels)

label_mapping = {label: idx for idx, label in enumerate(y.unique())}
y_encoded = y.map(label_mapping)

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

svm_model = SVC(kernel='linear', random_state=42, probability=True)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_val)
y_pred_prob = svm_model.predict_proba(X_val)

class_report = classification_report(y_val, y_pred, target_names=label_mapping.keys(), output_dict=True)

roc_auc = roc_auc_score(y_val, y_pred_prob, multi_class='ovr', average='macro')

balanced_accuracy = balanced_accuracy_score(y_val, y_pred)

class_report_tab = []
for label, metrics in class_report.items():
    if label == 'accuracy':
        continue
    class_report_tab.append([label] + [metrics['precision'], metrics['recall'], metrics['f1-score'], metrics['support']])

summary_tab = [
    ['ROC-AUC Score', roc_auc],
    ['Balanced Accuracy Score', balanced_accuracy]
]

print("\nClassification Report:")
print(tabulate(class_report_tab, headers=["Class", "Precision", "Recall", "F1-Score", "Support"], tablefmt="pretty"))

print("\nModel Evaluation Summary:")
print(tabulate(summary_tab, headers=["Metric", "Score"], tablefmt="pretty"))



Classification Report:
+--------------------------------------------------------+--------------------+--------------------+--------------------+---------+
|                         Class                          |     Precision      |       Recall       |      F1-Score      | Support |
+--------------------------------------------------------+--------------------+--------------------+--------------------+---------+
|                         Normal                         | 0.9615384615384616 |        1.0         | 0.9803921568627451 |  100.0  |
|                Complex_Partial_Seizures                |        1.0         |        0.97        | 0.9847715736040609 |  100.0  |
|                Electrographic_Seizures                 |        1.0         |        0.98        |  0.98989898989899  |  100.0  |
| Video_detected_Seizures_with_no_visual_change_over_EEG | 0.9801980198019802 |        0.99        | 0.9850746268656716 |  100.0  |
|                       macro avg                   